In [1]:
import numpy as np
import pandas as pd
import warnings

In [2]:
sales_train = pd.read_csv('./data/sales_train.csv')
shops = pd.read_csv('./data/shops.csv')
items = pd.read_csv('./data/items.csv')
item_categories = pd.read_csv('./data/item_categories.csv')
test = pd.read_csv('./data/test.csv')

In [3]:
print(sales_train.shape)
print(shops.shape)
print(items.shape)
print(item_categories.shape)
print(test.shape)

(2935849, 6)
(60, 2)
(22170, 3)
(84, 2)
(214200, 3)


In [4]:
sales_train.head()  # 월별 판매량만 구하려면 data 피처를 제거
                    # Target 값 = data_block_num 피처의 값이 같은 데이터들의 item_cnt_data값의 합임.
                    # 훈련 데이터 중 가장 최근인 2015년 10월 판매 내역을 검증 데이터로 사용

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [5]:
sales_train.info()  # 메모리 절감 전략이 필요함.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


In [6]:
shops.head()  # 상점 데이터 shop_name 의 첫 단어는 도시 이름임.
              # train_sale의 shop_id를 기준으로 sales_train과 shops 데이터 병합 필요

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [7]:
shops.info()   # 상점 데이터 60개로 결측 데이터가 없음.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   shop_name  60 non-null     object
 1   shop_id    60 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB


In [8]:
items.head()  # item 데이터에서 item_category_id 제거
              # item_id를 기준으로 sales_train과 items 병합

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [9]:
items.info()  # item 데이터는 결측 데이터가 없음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22170 entries, 0 to 22169
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   item_name         22170 non-null  object
 1   item_id           22170 non-null  int64 
 2   item_category_id  22170 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 519.7+ KB


In [10]:
item_categories.head()   # item_category_id를 기준으로 sales_train과 item_categories 데이터 병합 필요
                         # item_category_name의 첫 단어는 대분류로 분류하여야 함

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [11]:
item_categories.info()  # 모든 데이터의 결측값 없음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   item_category_name  84 non-null     object
 1   item_category_id    84 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.4+ KB


In [12]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


# 01. 피처명 한글화

## 01. sales_train

In [13]:
sales_train = sales_train.rename(columns={'date': '날짜',
                                          'date_block_num': '월ID',
                                          'shop_id': '상점ID',
                                          'item_id': '상품ID',
                                          'item_price': '판매가',
                                          'item_cnt_day': '판매량'})

In [14]:
sales_train.head()

,날짜,월ID,상점ID,상품ID,판매가,판매량
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


## 02. shop

In [15]:
shops = shops.rename(columns={'shop_name': '상점명',
                              'shop_id': '상점ID'})

In [16]:
shops.head()

,상점명,상점ID
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


## 03. items

In [17]:
items = items.rename(columns={'item_name': '상품명',
                              'item_id': '상품ID',
                              'item_category_id': '상품분류ID'})

In [18]:
items.head()

,상품명,상품ID,상품분류ID
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


## 04. item_categories

In [19]:
item_categories = item_categories.rename(columns=
                                         {'item_category_name': '상품분류명',
                                          'item_category_id': '상품분류ID'})

In [20]:
item_categories.head()

,상품분류명,상품분류ID
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


## 05. test

In [21]:
test = test.rename(columns={'shop_id': '상점ID',
                            'item_id': '상품ID'})

In [22]:
test.head()

,ID,상점ID,상품ID
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


# 02. 다운캐스팅이란?

* 다운캐스팅downcasting이란 더 작은 데이터 타입으로 변환하는 작업
* 금붕어는 금붕어용 어항에서. 돌고래용 수족관을 마련할 필요는 없다!
* 메모리 낭비를 막고, 훈련 속도를 빠르게 하려면 모든 데이터를 최대 타입으로 할당할 필요는 없음(분석 정리 5)

In [23]:
def downcast(df, verbose=True):
    start_mem=df.memory_usage().sum()/1024**2  # ( data fram을 가지고 있는 array 공간 차지 절감을 위해 )
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('{:.1f}% 압축됨'.format(100 * (start_mem - end_mem) / start_mem))
    return df

In [24]:
all_df = [sales_train, shops, items, item_categories, test]
for df in all_df:
    df = downcast(df)

62.5% 압축됨
38.6% 압축됨
54.2% 압축됨
39.9% 압축됨
70.8% 압축됨


In [25]:
all_df

[                 날짜  월ID  상점ID   상품ID          판매가  판매량
 0        02.01.2013    0    59  22154   999.000000    1
 1        03.01.2013    0    25   2552   899.000000    1
 2        05.01.2013    0    25   2552   899.000000   -1
 3        06.01.2013    0    25   2554  1709.050049    1
 4        15.01.2013    0    25   2555  1099.000000    1
 ...             ...  ...   ...    ...          ...  ...
 2935844  10.10.2015   33    25   7409   299.000000    1
 2935845  09.10.2015   33    25   7460   299.000000    1
 2935846  14.10.2015   33    25   7459   349.000000    1
 2935847  22.10.2015   33    25   7440   299.000000    1
 2935848  03.10.2015   33    25   7460   299.000000    1
 
 [2935849 rows x 6 columns],
                                                 상점명  상점ID
 0                     !Якутск Орджоникидзе, 56 фран     0
 1                     !Якутск ТЦ "Центральный" фран     1
 2                                  Адыгея ТЦ "Мега"     2
 3                    Балашиха ТРК "Октябрь-Кином

# 02. 피처 엔지니어링 II : 개별 데이터 피처 엔지니어링

## 2.1 sales_train 이상치 제거 및 전처리

* sales_train, shops, items, item_categories 데이터를 ‘각각’ 피처 엔지니어링
* sales_train 이상치 제거와 전처리
  - 판매가, 판매량이 음수라면 환불 건이거나 오류이므로 이상치로 간주
  - 또한, 판매가가 50,000 이상인 데이터, 판매량이 1,000 이상인 데이터도 이상치로 간주(분석 정리 9)

In [26]:
# 판매가가 0보다 큰 데이터 추출
sales_train = sales_train[sales_train['판매가'] > 0]

# 판매가가 50,000보다 작은 데이터 추출
sales_train = sales_train[sales_train['판매가'] < 50000]

# 판매량이 0보다 큰 데이터 추출
sales_train = sales_train[sales_train['판매량'] > 0]

# 판매량이 1,000보다 작은 데이터 추출
sales_train = sales_train[sales_train['판매량'] < 1000]

* sales_train 이상치 제거와 전처리
 - 상점명을 조금 다르게 기입해서 같은 상점인데 따로 기록돼 있는 상점이 네 쌍 있음
 - 다음 상점명 네 쌍을 같은 의미로 간주

In [27]:
print(shops['상점명'][0], '||', shops['상점명'][57])
print(shops['상점명'][1], '||', shops['상점명'][58])
print(shops['상점명'][10], '||', shops['상점명'][11])
print(shops['상점명'][39], '||', shops['상점명'][40])

!Якутск Орджоникидзе, 56 фран || Якутск Орджоникидзе, 56
!Якутск ТЦ "Центральный" фран || Якутск ТЦ "Центральный"
Жуковский ул. Чкалова 39м? || Жуковский ул. Чкалова 39м²
РостовНаДону ТРК "Мегацентр Горизонт" || РостовНаДону ТРК "Мегацентр Горизонт" Островной


[참고] 상점ID는 0부터 차례로 번호가 매겨져 있어서 0번째 상점명 데이터는 상점ID가 0인 데이터와 같음

* 0 vs. 57
  - !Якутск Орджоникидзе, 56 фран
  - Якутск Орджоникидзе, 56
* 1 vs. 58
  - !Якутск ТЦ "Центральный" фран
  - Якутск ТЦ "Центральный"
* 10 vs. 11
  - Жуковский ул. Чкалова 39м?
  - Жуковский ул. Чкалова 39м²
* 39 vs. 40
  - РостовНаДону ТРК "Мегацентр Горизонт"
  - РостовНаДону ТРК "Мегацентр Горизонт" Островной

### 상점명은 그대로 두고 상점ID만 수정
  - 상점명과 상점ID가 결국 1:1 매칭되고, 상점명은 문자 데이터라 나중에 제거할 예정이기 때문!

In [28]:
# sales_train 데이터에서 상점ID 수정
sales_train.loc[sales_train['상점ID'] == 0, '상점ID'] = 57
sales_train.loc[sales_train['상점ID'] == 1, '상점ID'] = 58
sales_train.loc[sales_train['상점ID'] == 10, '상점ID'] = 11
sales_train.loc[sales_train['상점ID'] == 39, '상점ID'] = 40

# test 데이터에서 상점ID 수정
test.loc[test['상점ID'] == 0, '상점ID'] = 57
test.loc[test['상점ID'] == 1, '상점ID'] = 58
test.loc[test['상점ID'] == 10, '상점ID'] = 11
test.loc[test['상점ID'] == 39, '상점ID'] = 40

## 2.2 shops 파생 피처 생성 및 인코딩

상점명의 첫 단어는 상점이 위치한 도시(분석 정리 6)

In [29]:
shops['도시'] = shops['상점명'].apply(lambda x: x.split()[0])
shops['도시'].unique()

array(['!Якутск', 'Адыгея', 'Балашиха', 'Волжский', 'Вологда', 'Воронеж',
       'Выездная', 'Жуковский', 'Интернет-магазин', 'Казань', 'Калуга',
       'Коломна', 'Красноярск', 'Курск', 'Москва', 'Мытищи', 'Н.Новгород',
       'Новосибирск', 'Омск', 'РостовНаДону', 'СПб', 'Самара', 'Сергиев',
       'Сургут', 'Томск', 'Тюмень', 'Уфа', 'Химки', 'Цифровой', 'Чехов',
       'Якутск', 'Ярославль'], dtype=object)

- !Якутск 앞의 느낌표(!)는 잘못 기재된 것이니 제거

In [30]:
shops.loc[shops['도시'] =='!Якутск', '도시'] = 'Якутск'

* 도시명 레이블 인코딩
  - 도시명은 범주형 피처임
  - 머신러닝 모델은 문자를 인식하지 못하므로 숫자로 바꿔야 함

In [31]:
from sklearn.preprocessing import LabelEncoder

# 레이블 인코더 생성
label_encoder = LabelEncoder()

# 도시 피처 레이블 인코딩
shops['도시'] = label_encoder.fit_transform(shops['도시'])

In [32]:
shops

,상점명,상점ID,도시
0,"!Якутск Орджоникидзе, 56 фран",0,29
1,"!Якутск ТЦ ""Центральный"" фран",1,29
2,"Адыгея ТЦ ""Мега""",2,0
3,"Балашиха ТРК ""Октябрь-Киномир""",3,1
4,"Волжский ТЦ ""Волга Молл""",4,2
5,"Вологда ТРЦ ""Мармелад""",5,3
6,"Воронеж (Плехановская, 13)",6,4
7,"Воронеж ТРЦ ""Максимир""",7,4
8,"Воронеж ТРЦ Сити-Парк ""Град""",8,4
9,Выездная Торговля,9,5


## 2.3 shops 파생 피처 제거

* 상점명 피처 제거
  - 상점명 피처는 도시명 피처를 만들기 위해 필요했음
  - 이젠 상점명 피처가 더는 필요 없으니 제거(같은 의미가 상점ID 피처에 내포돼 있기 때문)

In [33]:
# 상점명 피처 제거
shops = shops.drop('상점명', axis=1)

shops.head()

,상점ID,도시
0,0,29
1,1,29
2,2,0
3,3,1
4,4,2


* items 파생 피처 생성
  - 상품명 피처 제거
  - 우선, items 데이터는 상품명, 상품ID, 상품분류ID를 피처로 갖음(상품명은 상품ID와 일대일 매칭되므로 제거(분석 정리8))

In [34]:
# 상품명 피처 제거
items = items.drop(['상품명'], axis=1)

* 첫 판매월 피처 생성
  - 판매 내역 데이터(sales_train)에서 상품ID가 가장 처음 등장한 날의 월ID를 구하면 됨
  - 첫 판매월 피처는 모델링에 직접 사용하지 않고, 추후 다른 피처를 만들 때 활용됨

In [35]:
# 상품이 맨 처음 팔린 날을 피처로 추가
items['첫 판매월'] = sales_train.groupby('상품ID').agg({'월ID': 'min'})['월ID']

items.head()

,상품ID,상품분류ID,첫 판매월
0,0,40,20.0
1,1,76,15.0
2,2,40,19.0
3,3,40,18.0
4,4,40,20.0


* 첫 판매월 결측값 처리

In [36]:
items[items['첫 판매월'].isna()]

,상품ID,상품분류ID,첫 판매월
83,83,40,NaN
140,140,45,NaN
168,168,44,NaN
173,173,45,NaN
204,204,44,NaN
...,...,...,...
21974,21974,61,NaN
21975,21975,61,NaN
22022,22022,40,NaN
22035,22035,40,NaN


In [37]:
# 첫 판매월 피처의 결측값을 34로 대체
items['첫 판매월'] = items['첫 판매월'].fillna(34)

## 2.4 item_categories 파생 피처 생성 및 인코딩

*  item_categories 파생 피처 생성 및 인코딩
  - item_categories에서 ‘대분류’ 파생 피처 만들고, 이를 인코딩
  - 상품분류명의 첫 단어가 범주 대분류, 이를 참고해 대분류 추출(분석 정리 6)
  - 범주형 데이터를 더 큰 범주로 묶으면 범주가 지나치게 세밀할 대보다 성능 향상에 유리할 수도 있음

In [38]:
# 상품분류명의 첫 단어를 대분류로 추출
item_categories['대분류'] = item_categories['상품분류명'].apply(lambda x: x.split()[0])

* item_categories 파생 피처 생성 및 인코딩

In [39]:
item_categories['대분류'].value_counts()

Игры          14
Книги         13
Подарки       12
Игровые        8
Аксессуары     7
Музыка         6
Программы      6
Карты          5
Кино           5
Служебные      2
Чистые         2
PC             1
Билеты         1
Доставка       1
Элементы       1
Name: 대분류, dtype: int64

* 고윳값 5개 미만인 대분류는 ‘etc’로 변환
   - 대분류 하나가 범주를 일정 개수 이상을 갖는 게 성능 향상에 유리하기 때문

In [40]:
def make_etc(x):
    if len(item_categories[item_categories['대분류']==x]) >= 5:
        return x
    else:
        return 'etc'

# 대분류의 고윳값 개수가 5개 미만이면 'etc'로 바꾸기
item_categories['대분류'] = item_categories['대분류'].apply(make_etc)

In [41]:
item_categories.head()

,상품분류명,상품분류ID,대분류
0,PC - Гарнитуры/Наушники,0,etc
1,Аксессуары - PS2,1,Аксессуары
2,Аксессуары - PS3,2,Аксессуары
3,Аксессуары - PS4,3,Аксессуары
4,Аксессуары - PSP,4,Аксессуары


* 범주형 피처인 대분류 인코딩 후, 더 이상 필요 없는 상품분류명 피처 제거

In [42]:
# 레이블 인코더 생성
label_encoder = LabelEncoder()

# 대분류 피처 레이블 인코딩
item_categories['대분류'] = label_encoder.fit_transform(item_categories['대분류'])

# 상품분류명 피처 제거
item_categories = item_categories.drop('상품분류명', axis=1)

# 3. 작업 결과 정리

1. sales_train
  - 이상치 제거
  - 상품ID 네 쌍 수정

2. shops
  - 상점명 피처를 활용해 도시 피처 새로 추가
  - 범주형 데이터인 도시 피처 인코딩
  - 상점명 피처 제거

3. items
  - 첫 판매월 피처 추가
  - 결측값을 34로 대체

4. item_categories
 - 대분류 피처 추가
 - 고윳값 개수 5개 미만인 대분류 값을 ‘etc’로 변경
 - 대분류 피처 인코딩

Cf_  etc : 기타, 참조

# 04. 데이터 조합
  - Base Line과 동일

In [43]:
from itertools import product

In [44]:
train = []

In [45]:
for i in sales_train['월ID'].unique():
    all_shop = sales_train.loc[sales_train['월ID']==i, '상점ID'].unique()
    all_item = sales_train.loc[sales_train['월ID']==i, '상품ID'].unique()
    train.append(np.array(list(product([i], all_shop, all_item))))

idx_features = ['월ID', '상점ID', '상품ID'] # 기준 피처

# 리스트 타입인 train을 DataFrame 타입으로 변환
train = pd.DataFrame(np.vstack(train), columns=idx_features)

In [46]:
train

,월ID,상점ID,상품ID
0,0,59,22154
1,0,59,2552
2,0,59,2554
3,0,59,2555
4,0,59,2564
...,...,...,...
10812763,33,21,7635
10812764,33,21,7638
10812765,33,21,7640
10812766,33,21,7632


# 05. 데이터 조합 및 파생 피처 생성
* 파생 피처 생성
  - 3가지 피처 생성(‘월간 판매량’, ‘평균 판매가’, ‘기준 피처별 상품 판매건수’)
  - 먼저, 월간 판매량과 평균 판매가 피처 생성 (베이스라인과 다르게 평균 판매가도 추가)

In [47]:
group = sales_train.groupby(idx_features).agg({'판매량': 'sum',
                                               '판매가': 'mean'})
group = group.reset_index()
group = group.rename(columns={'판매량': '월간 판매량', '판매가': '평균 판매가'})

train = train.merge(group, on=idx_features, how='left')

train.head()

,월ID,상점ID,상품ID,월간 판매량,평균 판매가
0,0,59,22154,1.0,999.0
1,0,59,2552,NaN,NaN
2,0,59,2554,NaN,NaN
3,0,59,2555,NaN,NaN
4,0,59,2564,NaN,NaN


In [48]:
import gc

# group 변수 가비지 컬렉션
del group
gc.collect();

In [49]:
# 상품 판매건수 피처 추가
group = sales_train.groupby(idx_features).agg({'판매량': 'count'})
group = group.reset_index()
group = group.rename(columns={'판매량': '판매건수'})

train = train.merge(group, on=idx_features, how='left')

# 가비지 컬렉션
del group, sales_train
gc.collect()

train.head()

,월ID,상점ID,상품ID,월간 판매량,평균 판매가,판매건수
0,0,59,22154,1.0,999.0,1.0
1,0,59,2552,NaN,NaN,NaN
2,0,59,2554,NaN,NaN,NaN
3,0,59,2555,NaN,NaN,NaN
4,0,59,2564,NaN,NaN,NaN


# 06. 데이터 합치기

## 6.1 테스트 데이터 이어붙이기

* 지금까지 한 작업
* 월ID, 상점ID, 상품ID 조합으로 train을 만들고, 여기에 월간 판매량(타깃값)을 추가함
* 테스트 데이터 이어붙이기
  - 뒤이어 shops, items, item_categories 데이터를 병합할 텐데, 이때 테스트 데이터에도
    한 번에 병합하는 게 좋기 때문 우선, 월ID 피처 추가
    (2015년 10월의 월ID가 33이므로 2015년 11월의 월ID는 34)

In [50]:
test['월ID'] = 34

# train과 test 이어붙이기
all_data = pd.concat([train, test.drop('ID', axis=1)],
                     ignore_index=True, # 기존 인덱스 무시(0부터 새로 시작)
                     keys=idx_features) # 이어붙이는 기준이 되는 피처

In [51]:
all_data

,월ID,상점ID,상품ID,월간 판매량,평균 판매가,판매건수
0,0,59,22154,1.0,999.0,1.0
1,0,59,2552,NaN,NaN,NaN
2,0,59,2554,NaN,NaN,NaN
3,0,59,2555,NaN,NaN,NaN
4,0,59,2564,NaN,NaN,NaN
...,...,...,...,...,...,...
11026963,34,45,18454,NaN,NaN,NaN
11026964,34,45,16188,NaN,NaN,NaN
11026965,34,45,15757,NaN,NaN,NaN
11026966,34,45,19648,NaN,NaN,NaN


Cf_
- 2013년 1월 ~ 12월 : 0 ~ 11,
- 2014년 1월 ~ 12월 : 12 ~ 23,
- 2015년 1월 ~ 10월 : 24 ~ 33까지 index를 부여하고
- 2015년11월은 34번 index를 부여한 것임.
- 0 ~ 34번까지 Index를 부여한 사유는 컴퓨터는 년, 월, 일을 알지 못하므로, 0 ~ 34번까지 순차적으로 index를 부여하여 34번째 달의 판매량을 예측하는 것

- 결측값은 0으로 대체
  - train과 group을 병합할 때 결측값이 많았기 때문에

In [53]:
# 결측값을 0으로 대체
all_data = all_data.fillna(0)
all_data

,월ID,상점ID,상품ID,월간 판매량,평균 판매가,판매건수
0,0,59,22154,1.0,999.0,1.0
1,0,59,2552,0.0,0.0,0.0
2,0,59,2554,0.0,0.0,0.0
3,0,59,2555,0.0,0.0,0.0
4,0,59,2564,0.0,0.0,0.0
...,...,...,...,...,...,...
11026963,34,45,18454,0.0,0.0,0.0
11026964,34,45,16188,0.0,0.0,0.0
11026965,34,45,15757,0.0,0.0,0.0
11026966,34,45,19648,0.0,0.0,0.0


## 6.2 나머지 데이터 병합(최종 데이터 생성)

- 나머지 데이터를 all_data에 병합(분석 정리 4)
- 추가로, 데이터 다운캐스팅과 가비지 컬렉션 수행(분석 정리 5)

In [54]:
# 나머지 데이터 병합
all_data = all_data.merge(shops, on='상점ID', how='left')
all_data = all_data.merge(items, on='상품ID', how='left')
all_data = all_data.merge(item_categories, on='상품분류ID', how='left')

# 데이터 다운캐스팅
all_data = downcast(all_data)

# 가비지 컬렉션
del shops, items, item_categories
gc.collect();

59.6% 압축됨


## 6.3 병합한 데이터 확인

In [55]:
all_data.head()

,월ID,상점ID,상품ID,월간 판매량,평균 판매가,판매건수,도시,상품분류ID,첫 판매월,대분류
0,0,59,22154,1,999.0,1,30,37,0,5
1,0,59,2552,0,0.0,0,30,58,0,7
2,0,59,2554,0,0.0,0,30,58,0,7
3,0,59,2555,0,0.0,0,30,56,0,7
4,0,59,2564,0,0.0,0,30,59,0,7


# 07. 시차 피처 생성
  - 기준 피처별 월간 평균 판매량 파생 피처 생성

In [56]:
def add_mean_features(df, mean_features, idx_features):
    # 기준 피처 확인 
    assert (idx_features[0] == '월ID') and \
           len(idx_features) in [2, 3]
    
    # 파생 피처명 설정 
    if len(idx_features) == 2:
        feature_name = idx_features[1] + '별 평균 판매량'
    else:
        feature_name = idx_features[1] + ' ' + idx_features[2] + '별 평균 판매량'
    
    # 기준 피처를 토대로 그룹화해 월간 평균 판매량 구하기 
    group = df.groupby(idx_features).agg({'월간 판매량': 'mean'})
    group = group.reset_index()
    group = group.rename(columns={'월간 판매량': feature_name})
    
    # df와 group 병합 
    df = df.merge(group, on=idx_features, how='left')
    # 데이터 다운캐스팅 
    df = downcast(df, verbose=False)
    # 새로 만든 feature_name 피처명을 mean_features 리스트에 추가 
    mean_features.append(feature_name)
    
    # 가비지 컬렉션
    del group
    gc.collect()
    
    return df, mean_features

In [57]:
# 그룹화 기준 피처 중 '상품ID'가 포함된 파생 피처명을 담을 리스트
item_mean_features = []

# ['월ID', '상품ID']로 그룹화한 월간 평균 판매량 파생 피처 생성
all_data, item_mean_features = add_mean_features(df=all_data,
                                                 mean_features=item_mean_features,
                                                 idx_features=['월ID', '상품ID'])

# ['월ID', '상품ID', '도시']로 그룹화한 월간 평균 판매량 파생 피처 생성
all_data, item_mean_features = add_mean_features(df=all_data,
                                                 mean_features=item_mean_features,
                                                 idx_features=['월ID', '상품ID', '도시'])

In [58]:
item_mean_features

['상품ID별 평균 판매량', '상품ID 도시별 평균 판매량']

In [59]:
# 그룹화 기준 피처 중 '상점ID'가 포함된 파생 피처명을 담을 리스트
shop_mean_features = []

# ['월ID', '상점ID', '상품분류ID']로 그룹화한 월간 평균 판매량 파생 피처 생성
all_data, shop_mean_features = add_mean_features(df=all_data, 
                                                 mean_features=shop_mean_features,
                                                 idx_features=['월ID', '상점ID', '상품분류ID'])

In [60]:
shop_mean_features

['상점ID 상품분류ID별 평균 판매량']

## 7.1 시차 피처 생성 원리와 구현

In [61]:
def add_lag_features(df, lag_features_to_clip, idx_features, 
                     lag_feature, nlags=3, clip=False):
    # 시차 피처 생성에 필요한 DataFrame 부분만 복사 
    df_temp = df[idx_features + [lag_feature]].copy() 

    # 시차 피처 생성 
    for i in range(1, nlags+1):
        # 시차 피처명 
        lag_feature_name = lag_feature +'_시차' + str(i)
        # df_temp 열 이름 설정 
        df_temp.columns = idx_features + [lag_feature_name]
        # df_temp의 date_block_num 피처에 1 더하기 
        df_temp['월ID'] += 1
        # idx_feature를 기준으로 df와 df_temp 병합하기 
        df = df.merge(df_temp.drop_duplicates(), 
                      on=idx_features, 
                      how='left')
        # 결측값 0으로 대체 
        df[lag_feature_name] = df[lag_feature_name].fillna(0)
        # 0 ~ 20 사이로 제한할 시차 피처명을 lag_features_to_clip에 추가 
        if clip: 
            lag_features_to_clip.append(lag_feature_name)
    
    # 데이터 다운캐스팅
    df = downcast(df, False)
    # 가비지 컬렉션
    del df_temp
    gc.collect()
    
    return df, lag_features_to_clip

### 시차 피처 생성 I : 월간 판매량

In [62]:
lag_features_to_clip = [] # 0 ~ 20 사이로 제한할 시차 피처명을 담을 리스트
idx_features = ['월ID', '상점ID', '상품ID'] # 기준 피처

# idx_features를 기준으로 월간 판매량의 세 달치 시차 피처 생성
all_data, lag_features_to_clip = add_lag_features(df=all_data, 
                                                  lag_features_to_clip=lag_features_to_clip,
                                                  idx_features=idx_features,
                                                  lag_feature='월간 판매량', 
                                                  nlags=3,
                                                  clip=True) # 값을 0 ~ 20 사이로 제한

In [63]:
all_data.head().T

,0,1,2,3,4
월ID,0.000000,0.000000,0.000000,0.000000,0.000000
상점ID,59.000000,59.000000,59.000000,59.000000,59.000000
상품ID,22154.000000,2552.000000,2554.000000,2555.000000,2564.000000
월간 판매량,1.000000,0.000000,0.000000,0.000000,0.000000
평균 판매가,999.000000,0.000000,0.000000,0.000000,0.000000
판매건수,1.000000,0.000000,0.000000,0.000000,0.000000
도시,30.000000,30.000000,30.000000,30.000000,30.000000
상품분류ID,37.000000,58.000000,58.000000,56.000000,59.000000
첫 판매월,0.000000,0.000000,0.000000,0.000000,0.000000
대분류,5.000000,7.000000,7.000000,7.000000,7.000000


In [64]:
lag_features_to_clip

['월간 판매량_시차1', '월간 판매량_시차2', '월간 판매량_시차3']

### 시차 피처 생성 II : 판매건수, 평균 판매가

In [65]:
# idx_features를 기준으로 판매건수 피처의 세 달치 시차 피처 생성
all_data, lag_features_to_clip = add_lag_features(df=all_data, 
                                                  lag_features_to_clip=lag_features_to_clip,
                                                  idx_features=idx_features,
                                                  lag_feature='판매건수', 
                                                  nlags=3)

# idx_features를 기준으로 평균 판매가 피처의 세 달치 시차 피처 생성
all_data, lag_features_to_clip = add_lag_features(df=all_data, 
                                                  lag_features_to_clip=lag_features_to_clip,
                                                  idx_features=idx_features,
                                                  lag_feature='평균 판매가', 
                                                  nlags=3)

### 시차 피처 생성 III : 평균 판매량

In [66]:
# idx_features를 기준으로 item_mean_features 요소별 시차 피처 생성
for item_mean_feature in item_mean_features:
    all_data, lag_features_to_clip = add_lag_features(df=all_data, 
                                                      lag_features_to_clip=lag_features_to_clip, 
                                                      idx_features=idx_features, 
                                                      lag_feature=item_mean_feature, 
                                                      nlags=3,
                                                      clip=True)
# item_mean_features 피처 제거
all_data = all_data.drop(item_mean_features, axis=1)

In [67]:
lag_features_to_clip

['월간 판매량_시차1',
 '월간 판매량_시차2',
 '월간 판매량_시차3',
 '상품ID별 평균 판매량_시차1',
 '상품ID별 평균 판매량_시차2',
 '상품ID별 평균 판매량_시차3',
 '상품ID 도시별 평균 판매량_시차1',
 '상품ID 도시별 평균 판매량_시차2',
 '상품ID 도시별 평균 판매량_시차3']

In [68]:
# ['월ID', '상점ID', '상품분류ID']를 기준으로 shop_mean_features 요소별 시차 피처 생성
for shop_mean_feature in shop_mean_features:
    all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                      lag_features_to_clip=lag_features_to_clip,
                                                      idx_features=['월ID', '상점ID', '상품분류ID'],
                                                      lag_feature=shop_mean_feature,
                                                      nlags=3,
                                                      clip=True)
# shop_mean_features 피처 제거
all_data = all_data.drop(shop_mean_features, axis=1)

## 7.2 시차 피처 생성 마무리 : 결측값 처리

In [69]:
# 월ID 3미만인 데이터 제거
all_data = all_data.drop(all_data[all_data['월ID'] < 3].index)

# 08. 피처 엔지니어링 VI : 기타 피처 엔지니어링

## 8.1 월간 판매량 시차 피처들의 평균

In [70]:
all_data['월간 판매량 시차평균'] = all_data[['월간 판매량_시차1',
                                          '월간 판매량_시차2', 
                                          '월간 판매량_시차3']].mean(axis=1)

In [71]:
# 0 ~ 20 사이로 값 제한
all_data[lag_features_to_clip + ['월간 판매량', '월간 판매량 시차평균']] = all_data[lag_features_to_clip + ['월간 판매량', '월간 판매량 시차평균']].clip(0, 20)

## 8.2 시차 변화량

In [72]:
all_data['시차변화량1'] = all_data['월간 판매량_시차1']/all_data['월간 판매량_시차2']
all_data['시차변화량1'] = all_data['시차변화량1'].replace([np.inf, -np.inf], 
                                                        np.nan).fillna(0)

all_data['시차변화량2'] = all_data['월간 판매량_시차2']/all_data['월간 판매량_시차3']
all_data['시차변화량2'] = all_data['시차변화량2'].replace([np.inf, -np.inf], 
                                                        np.nan).fillna(0)

## 8.3 신상 여부

In [73]:
all_data['신상여부'] = all_data['첫 판매월'] == all_data['월ID']

## 8.4 첫 판매 후 경과 기간

In [74]:
all_data['첫 판매 후 기간'] = all_data['월ID'] - all_data['첫 판매월']

## 8.5 월(month)

In [75]:
all_data['월'] = all_data['월ID'] % 12

## 8.6 필요 없는 피처 제거

In [76]:
# 첫 판매월, 평균 판매가, 판매건수 피처 제거
all_data = all_data.drop(['첫 판매월', '평균 판매가', '판매건수'], axis=1)

In [77]:
all_data = downcast(all_data, False) # 데이터 다운캐스팅

# 09. 피처 엔지니어링 VII : 마무리

In [78]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9904582 entries, 1122386 to 11026967
Data columns (total 31 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   월ID                      int8   
 1   상점ID                     int8   
 2   상품ID                     int16  
 3   월간 판매량                   int8   
 4   도시                       int8   
 5   상품분류ID                   int8   
 6   대분류                      int8   
 7   월간 판매량_시차1               int8   
 8   월간 판매량_시차2               int8   
 9   월간 판매량_시차3               int8   
 10  판매건수_시차1                 int8   
 11  판매건수_시차2                 int8   
 12  판매건수_시차3                 int8   
 13  평균 판매가_시차1               float32
 14  평균 판매가_시차2               float32
 15  평균 판매가_시차3               float32
 16  상품ID별 평균 판매량_시차1         float32
 17  상품ID별 평균 판매량_시차2         float32
 18  상품ID별 평균 판매량_시차3         float32
 19  상품ID 도시별 평균 판매량_시차1      float32
 20  상품ID 도시별 평균 판매량_시차2      float32
 21  상

In [79]:
# 훈련 데이터 (피처)
X_train = all_data[all_data['월ID'] < 33]
X_train = X_train.drop(['월간 판매량'], axis=1)

# 검증 데이터 (피처)
X_valid = all_data[all_data['월ID'] == 33]
X_valid = X_valid.drop(['월간 판매량'], axis=1)

# 테스트 데이터 (피처)
X_test = all_data[all_data['월ID'] == 34]
X_test = X_test.drop(['월간 판매량'], axis=1)

# 훈련 데이터 (타깃값)
y_train = all_data[all_data['월ID'] < 33]['월간 판매량']

# 검증 데이터 (타깃값)
y_valid = all_data[all_data['월ID'] == 33]['월간 판매량']

# 가비지 컬렉션
del all_data
gc.collect();

# 10. 모델 훈련 및 성능 검증

In [80]:
import lightgbm as lgb

# LightGBM 하이퍼파라미터
params = {'metric': 'rmse',
          'num_leaves': 255,
          'learning_rate': 0.005,
          'feature_fraction': 0.75,
          'bagging_fraction': 0.75,
          'bagging_freq': 5,
          'force_col_wise': True,
          'random_state': 10}
cat_features = ['상점ID', '도시', '상품분류ID', '대분류', '월']

# LightGBM 훈련 및 검증 데이터셋
dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid)

# LightGBM 모델 훈련
lgb_model = lgb.train(params=params,
                      train_set=dtrain,
                      num_boost_round=1500,
                      valid_sets=(dtrain, dvalid),
                      early_stopping_rounds=150,
                      categorical_feature=cat_features,
                      verbose_eval=100)

C:\Users\KOREAIT\.conda\envs\py39tf2\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\KOREAIT\.conda\envs\py39tf2\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['대분류', '도시', '상점ID', '상품분류ID', '월']
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\KOREAIT\.conda\envs\py39tf2\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM.

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Total Bins 3902
[LightGBM] [Info] Number of data points in the train set: 9452298, number of used features: 30


C:\Users\KOREAIT\.conda\envs\py39tf2\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\KOREAIT\.conda\envs\py39tf2\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 0.297707
Training until validation scores don't improve for 150 rounds
[100]	training's rmse: 1.01073	valid_1's rmse: 0.986993
[200]	training's rmse: 0.909083	valid_1's rmse: 0.922887
[300]	training's rmse: 0.857505	valid_1's rmse: 0.898233
[400]	training's rmse: 0.829175	valid_1's rmse: 0.888136
[500]	training's rmse: 0.810627	valid_1's rmse: 0.884038
[600]	training's rmse: 0.796874	valid_1's rmse: 0.882521
[700]	training's rmse: 0.786815	valid_1's rmse: 0.882318
[800]	training's rmse: 0.778893	valid_1's rmse: 0.882091
[900]	training's rmse: 0.772392	valid_1's rmse: 0.881891
[1000]	training's rmse: 0.76691	valid_1's rmse: 0.881599
[1100]	training's rmse: 0.76187	valid_1's rmse: 0.881711
Early stopping, best iteration is:
[999]	training's rmse: 0.766974	valid_1's rmse: 0.881574


# 예측 및 결과 제출

In [81]:
# 예측
preds = lgb_model.predict(X_test).clip(0, 20)
preds

# 제출 파일 생성
# submission['item_cnt_month'] = preds
# submission.to_csv('submission.csv', index=False)

array([0.41387411, 0.49971929, 0.80369473, ..., 0.0458018 , 0.01741607,
       0.04287869])

In [82]:
del X_train, y_train, X_valid, y_valid, X_test, lgb_model, dtrain, dvalid
gc.collect();